<a href="https://colab.research.google.com/github/CarloCHEN/Tweet-Sentiment-Extraction_Kaggle/blob/master/Phrase_extraction_by_considering_sentiment_(negative).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from nltk.tokenize import sent_tokenize

In [2]:
pip install pytorch_pretrained_bert

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

Using TensorFlow backend.


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1P4z6napWZDIZS7hFhnAL3GUYnxpz1aPS'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train_1.csv')  
train = pd.read_csv('train_1.csv')

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [8]:
device

device(type='cuda')

In [9]:
train[train['sentiment_ENG'] == 'negative']['text']

3               i dont think you can vote anymore! i tried
5                             headache  wanna see my Julie
12                                        Miss you my dear
15       Today Dan bought me Bio Dome AND the Reality B...
16                                   oo noo thats not good
                               ...                        
27460    stomach cramps; sat in bed with a hot water bo...
27462                                        my hair hurts
27468    save me--i feel like i'm going crazy.  ? http:...
27477    Omg I just woke up after only sleeping for 3hr...
27478                                      I can't keep it
Name: text, Length: 7786, dtype: object

In [0]:
sentences = train[train['sentiment_ENG'] == 'negative']['text']

In [0]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [12]:
max_len = 0
for i in range(len(tokenized_texts)):
    if (len(tokenized_texts[i]) > max_len):
        max_len = len(tokenized_texts[i])
    
max_len

93

In [0]:
selected = train[train['sentiment_ENG'] == 'negative']['selected_text']
selected_tokenized_texts = [tokenizer.tokenize(sent) for sent in selected]

In [14]:
selected_max_len = 0
for i in range(len(selected_tokenized_texts)):
    if (len(selected_tokenized_texts[i]) > selected_max_len):
        selected_max_len = len(selected_tokenized_texts[i])
    
selected_max_len

65

In [15]:
len(selected_tokenized_texts)

7786

In [0]:
x = np.arange(len(selected_tokenized_texts) * max_len)
x = x.reshape(len(selected_tokenized_texts), max_len)
x = np.zeros_like(x)
labels = x

for i in range(len(selected_tokenized_texts)):
  for selected_token in selected_tokenized_texts[i]:
    for j in range(len(tokenized_texts[i])):
      if (selected_token == tokenized_texts[i][j]):
        labels[i][j] = 1

In [0]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]


In [0]:
tr_inputs, val_inputs, tr_labels, val_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [0]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_labels = torch.tensor(tr_labels)
val_labels = torch.tensor(val_labels)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [20]:
tr_labels.shape

torch.Size([7007, 93])

In [0]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

valid_data = TensorDataset(val_inputs, val_masks, val_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=32)

In [0]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=2)
model = model.cuda()

In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [24]:
%pip install seqeval

In [0]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
tags_vals = ['absent', 'present']

In [27]:
epochs = 4
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []

    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.5720111271834265


Epoch:  25%|██▌       | 1/4 [01:27<04:21, 87.04s/it]

Validation loss: 0.5307001233100891
Validation Accuracy: 0.8077541544477027
F1-Score: 0.04318734793187348
Train loss: 0.4993935540386531


Epoch:  50%|█████     | 2/4 [02:53<02:54, 87.00s/it]

Validation loss: 0.5308351516723633
Validation Accuracy: 0.7318426197458457
F1-Score: 0.05339539978094194
Train loss: 0.40147408176230515


Epoch:  75%|███████▌  | 3/4 [04:20<01:26, 86.96s/it]

Validation loss: 0.6187839436531067
Validation Accuracy: 0.7575684261974587
F1-Score: 0.06286310144495755
Train loss: 0.3037951715188484


Epoch: 100%|██████████| 4/4 [05:47<00:00, 86.93s/it]

Validation loss: 0.7000952684879302
Validation Accuracy: 0.6343035190615837
F1-Score: 0.04582278481012659


In [28]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])

    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))


Validation loss: 0.7000952684879302
Validation Accuracy: 0.6343035190615837
Validation F1-Score: 0.0803793181305035


In [0]:
def keywordextract(sentence):
    predicted_phrase = []
    text = sentence
    tkns = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
    segments_ids = [0] * len(tkns)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    # print(tokens_tensor)
    # print(segments_tensors)
    model.eval()
    prediction = []
    logit = model(tokens_tensor, token_type_ids=None,
                                  attention_mask=segments_tensors)
    logit = logit.detach().cpu().numpy()
    # print(logit)
    prediction.extend([list(p) for p in np.argmax(logit, axis=2)])

    for k, j in enumerate(prediction[0]):
        if j==1 or j==0:
          pass
            # print(tokenizer.convert_ids_to_tokens(tokens_tensor[0].to('cpu').numpy())[k], j)
        if j == 1:
          predicted_phrase.append(tokenizer.convert_ids_to_tokens(tokens_tensor[0].to('cpu').numpy())[k])
    print("predicted phrase is: ")
    for element in predicted_phrase:  
      print(element, end =" ")


In [30]:
sentences

3               i dont think you can vote anymore! i tried
5                             headache  wanna see my Julie
12                                        Miss you my dear
15       Today Dan bought me Bio Dome AND the Reality B...
16                                   oo noo thats not good
                               ...                        
27460    stomach cramps; sat in bed with a hot water bo...
27462                                        my hair hurts
27468    save me--i feel like i'm going crazy.  ? http:...
27477    Omg I just woke up after only sleeping for 3hr...
27478                                      I can't keep it
Name: text, Length: 7786, dtype: object

In [32]:
keywordextract(sentences[3])
print(" ")
print("correct phrase is: " + selected[3])
print(" ")
keywordextract(sentences[27468])
print(" ")
print("correct phrase is: " + selected[27468])

predicted phrase is: 
i don ##t think you can vote anymore ! i tried  
correct phrase is: i dont think you can vote anymore!
 
predicted phrase is: 
save me - - i feel like i ' m going crazy . .  
correct phrase is: i feel like i'm going crazy.


In [33]:
# save model to google drive
# first, mount the drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# save the model with the defined name
model_save_name = 'BERT_for_negative_extraction.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [0]:
# re-load the model from the drive
# model_save_name = 'classifier.pt'
# path = F"/content/gdrive/My Drive/{model_save_name}"
# model.load_state_dict(torch.load(path))